In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
import os
import cPickle as pickle
from collections import Counter
import pandas
from datetime import datetime
from shutil import copyfile
import platform
import scipy.stats

In [2]:
popularRoot = "/datasets/sagarj/vineData/Dataset/"
recentRoot = "/datasets/sagarj/vine2016/Dataset/"
PostDir = "savedPosts/"
popUserDir = "Users/"
unpopUserDir = "UserProfiles3/"
vidsDir = "Videos/"
tracked = "/postTracking/Day53/"
unpopularDir = "/datasets/sagarj/UnPopular2016_senti/"
unpopVidsDir = unpopularDir + "Videos/"
unpopPostsDir = unpopularDir + "Posts/"
socialFeatures = "../Final_features/postSocialMetaFeatures_revised.pk"


In [3]:
def readJson(path):
    f = open(path)
    data = json.loads(f.read())
    return data

def getPopular():
    popularDir = popularRoot + PostDir
    popularPosts = os.listdir(popularDir)
    posts = []
    for post in popularPosts:
        record = readJson(popularDir + post)
        p = record['data']
        if isinstance(p,dict):
            posts.append(p['records'][0])
    return posts

def getUsersPop():
    popularDir = popularRoot + popUserDir
    popularPosts = os.listdir(popularDir)
    userMetaDict = {}
    posts = []
    followers = []
    for post in popularPosts:
        record = readJson(popularDir + post)
        p = record['data']
        if isinstance(p,dict) and p.has_key('followerCount'):
            posts.append(p)
            userId = p['userId']
            flwer= p['followerCount']
            pastPosts = p['authoredPostCount']
            followers.append(p['followerCount'])
            #userMetaDict[userId] = [flwer , pastPosts]
            userMetaDict[userId] = {'follower' : flwer , 'pastPosts': pastPosts}     
    return posts , followers , userMetaDict

def getUsersUnPop():
    popularDir = recentRoot + unpopUserDir
    popularPosts = os.listdir(popularDir)
    posts = []
    followers = []
    userMetaDict = {}
    for post in popularPosts:
        record = readJson(popularDir + post)
        p = record['data']
        if isinstance(p,dict) and p.has_key('followerCount'):
            posts.append(p)
            userId = p['userId']
            flwer = p['followerCount']
            pastPosts = p['authoredPostCount']
            followers.append(p['followerCount'])
            userMetaDict[userId] = {'follower' : flwer , 'pastPosts': pastPosts}          
    return posts , followers , userMetaDict

def sorted_ls(path):
    mtime = lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

    

def getCrawled(root , postDir):
    crawledDir = root + postDir
    crawledPosts = os.listdir(crawledDir)
    print len(crawledPosts)
    posts = []
    postDict = {}
    for post in crawledPosts:
        postid = post.split('.')[0]
        record = readJson(crawledDir + post)
        p = record['data']
        if isinstance(p,dict):
            posts.append(p['records'][0])
            postDict[postid] = p['records'][0]
    print len(postDict.keys())
    return posts , postDict

def getPopMetrics(vineList):
    likes = []
    reposts = []
    loops = []
    timestamps = []
    for vine in vineList:
        likes.append(vine['likes']['count'])
        reposts.append(vine['reposts']['count'])
        loops.append(vine['loops']['count'])
        timestamps.append(datetime.strptime(vine['created'], '%Y-%m-%dT%H:%M:%S.%f'))
    return likes, reposts, loops, timestamps

def copyList(listOfFiles,srcFolder , dstFolder):
    if not os.path.exists(dstFolder):
        os.makedirs(dstFolder)
    for f in listOfFiles:
        src = srcFolder + f
        dst = dstFolder + f
        copyfile(src , dst)
        
    sentiDict = {}
    for line in lines:
        comp = line.split(',')
        sentiDict[comp[0]] = float(comp[1].strip())
    return sentiDict

def readFeatureFile(color_features):
    with open(color_features) as g:
        featureLines = g.readlines()
    print len(featureLines)
    return featureLines


def postFollowerMapping(postsInterested , postDict , userDict ):
    finalMeta = []
    finalDict = {}
    for postId in postsInterested:
        if postId in postDict:
            post = postDict[postId]
            like = post['likes']['count']
            loop = post['loops']['count']
            reposts = post['reposts']['count']
            user = post['userId']
            if user in userDict:
                userMeta = userDict[user]
                logline = [postId,  userMeta['follower'], userMeta['pastPosts'] , loop , like , reposts ]
                finalMeta.append(logline)
                finalDict[postId] = {}
                
                finalDict[postId]['followers'] = userMeta['follower']
                finalDict[postId]['pastPosts'] = userMeta['pastPosts']
                finalDict[postId]['loops'] = loop
                finalDict[postId]['likes'] = like
                finalDict[postId]['reposts'] = reposts
    return finalMeta , finalDict
        
        
            
        
    


In [4]:
# print sorted_ls(unpopPostsDir)

In [5]:

recentList , recentDict = getCrawled(recentRoot , PostDir )

122326
122298


In [6]:
recentDict.keys()[1]

'1385887276084670464'

In [7]:
popularList = getPopular()

In [8]:
trackedList , trackedDict = getCrawled(recentRoot , tracked )

106396
74948


In [9]:
popList , popDict = getCrawled(popularRoot , PostDir )

11448
11448


In [32]:
popDict[popDict.keys()[1101]]

{u'avatarUrl': u'http://v.cdn.vine.co/r/avatars/C37F709C791330959094663778304_54444c9551b.9.0.jpg?versionId=NktgX9QkffjNkFEIxzFuz3kbyuljSQIb',
 u'blocked': 0,
 u'comments': {u'anchor': 1334421841082802176,
  u'anchorStr': u'1334421841082802176',
  u'backAnchor': u'',
  u'count': 433,
  u'nextPage': 2,
  u'previousPage': None,
  u'records': [{u'comment': u'Being in love is the same as being on drugs \U0001f44c',
    u'commentId': 1334421841082802176,
    u'created': u'2016-04-20T04:06:13.000000',
    u'entities': [],
    u'postId': 1271515766126260224,
    u'profileBackground': u'0x68bf60',
    u'user': {u'avatarUrl': u'http://v.cdn.vine.co/r/avatars/E6229947B71308934911952355328_489a9c3635c.5.1.jpg?versionId=zW_kefTzAgW_IotzfxlSIBtx6XDDUR4V',
     u'description': u'@anime.sickness',
     u'hideFromPopular': 0,
     u'location': u'',
     u'notPorn': 0,
     u'private': 0,
     u'profileBackground': u'0x68bf60',
     u'twitterVerified': 0,
     u'unflaggable': 0,
     u'userId': 1219913

In [34]:
userList = []
for d in popDict:
    if popDict[d]['username'] not in userList:
        uname = popDict[d]['username'].encode('utf8')
        out = uname.decode('unicode_escape').encode('ascii','ignore')
        userList.append(out)

In [35]:
print len(userList)

3306


In [36]:
import re
f = open('PopUsernames.txt' , 'wb')
for d in userList:
    line = d + '\n'
    f.write(line)
f.close()

In [ ]:
popList[100]

In [ ]:
recentLikes , recentReposts, recentLoops , recentTimestamps = getPopMetrics(recentList)

popLikes , popReposts, popLoops , popTimeStamps= getPopMetrics(popularList)

trackedLikes , trackedReposts, trackedLoops , trackedTimeStamps= getPopMetrics(trackedList)

In [ ]:
print np.min(popLikes) , np.min(popReposts) , np.min(popLoops)

In [ ]:
totalDict = trackedDict.copy()
totalDict.update(popDict)
print len(totalDict.keys())

In [ ]:
totalPostDict = dict(totalDict.items() + recentDict.items())

In [ ]:
print len(totalPostDict.keys())

In [ ]:
Likes = popLikes+ trackedLikes
Reposts = popReposts+ recentReposts

In [ ]:
Likes = map(lambda x: 0.000001 if x == 0 else x, Likes)
Reposts = map(lambda x: 0.000001 if x == 0 else x, Reposts)

In [ ]:
print np.mean(popLoops) , np.median(popLoops) , np.mean(popReposts) , np.median(popReposts) , np.mean(popLikes) , np.median(popLikes)

In [ ]:
print np.var(np.log(popLoops)),np.var(np.log(popLikes)),np.var(np.log(popReposts))


In [ ]:
print scipy.stats.pearsonr( popLoops , popLikes)
print scipy.stats.pearsonr( popReposts , popLikes)
print scipy.stats.pearsonr( popLoops , popReposts)

In [ ]:
sampleL = np.random.choice(len(Likes) , 3000)
sampleR = np.random.choice(len(Reposts) , 3000)

In [ ]:
likes = [Likes[i] for i in sampleL]
reposts = [Reposts[i] for i in sampleR]

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(np.log10(likes), 2000, normed=1,histtype='step', cumulative=True,linewidth = 3.0,linestyle='dashed')
plt.hist(np.log10(reposts), 2000, normed=1,histtype='step', cumulative=True,linewidth = 3.0)
plt.title("CDF for Likes and Repost", fontsize = 25)
plt.xlabel("Likes/repost count (Log scale)",fontsize = 25)
plt.ylabel("CDF",fontsize = 25)
plt.legend(['Likes' , 'Reposts'])
plt.grid(True)
plt.show()

In [ ]:
Top1000 = np.argsort(trackedLoops)[-50:]
trackedTop1000 = [trackedLoops[i] for i in Top1000]
trackedTop1000Posts = [trackedList[i] for i in Top1000]
print len(trackedTop1000)

In [ ]:
print np.median(trackedTop1000) , np.mean(trackedTop1000) , np.var(trackedTop1000)

In [ ]:
unpopular3000 = np.argsort(trackedLoops)[:12000]
unpop3000loops = [trackedLoops[i] for i in unpopular3000]
unpop3000Posts = [trackedList[i] for i in unpopular3000]

In [ ]:
print np.median(unpop3000loops) , np.mean(unpop3000loops) , np.var(unpop3000loops)

In [ ]:
unpopPostList = []
for p in unpop3000Posts: 
    postid = p['postId']
    filename = str(postid) + ".json"
    unpopPostList.append(filename)

In [ ]:
print len(unpopPostList)

In [ ]:
# srcPosts = "/datasets/sagarj/vine2016/Dataset//postTracking/Day24/"
# copyList(unpopPostList, srcPosts , unpopPostsDir)

In [ ]:
unpopVidList = []
for p in unpop3000Posts: 
    postid = p['videoUrl'].split('/')[5].split('?')[0]
    filename = str(postid)
    unpopVidList.append(filename)

In [ ]:
print unpopVidList[:10]

In [ ]:
# srcVids = recentRoot + vidsDir
# copyList(unpopVidList, srcVids , unpopVidsDir)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
#plt.hist(popLikes, 1000, normed=1,histtype='step', cumulative=True,linewidth = 3.0,linestyle='dashed')
#plt.hist(popReposts, 1000, normed=1,histtype='step', cumulative=False,linewidth = 3.0)
plt.hist(trackedTop1000, 20, normed=1,histtype='step', cumulative=True,linewidth = 3.0)
plt.title("CDF for Likes and Repost", fontsize = 25)
plt.xlabel("Likes/repost count (Log scale)",fontsize = 25)
plt.ylabel("CDF",fontsize = 25)
plt.legend(['Likes' , 'Reposts'])
plt.grid(True)
plt.show()

In [ ]:
print np.median(popLoops) , np.mean(popLoops) , np.var(popLoops)

In [ ]:
print pow(10,4.5)

In [ ]:
popUsers, popFollowers , popDict = getUsersPop()

In [ ]:
print np.mean(popFollowers) , np.median(popFollowers) , len(popFollowers)

In [ ]:
unpopUsers, unpopFollowers , unpopDict = getUsersUnPop()

In [ ]:
userDict = unpopDict.copy()
userDict.update(popDict)

In [ ]:
print len(userDict.keys())

In [ ]:
print unpopUsers[1]

In [ ]:
print np.mean(unpopFollowers) , np.median(unpopFollowers) , len(unpopFollowers)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(popFollowers, 100, normed=1,histtype='bar', cumulative=False,linewidth = 3.0)
#plt.hist(unpopFollowers, 100, normed=1,histtype='step', cumulative=True,linewidth = 3.0)
plt.title("CDF for Followers for Popular and unpopular posts", fontsize = 25)
plt.xlabel("Follower Count",fontsize = 25)
plt.ylabel("CDF",fontsize = 25)
plt.legend(['Popular' , 'Unpopular'])
plt.grid(True)
plt.show()

In [ ]:
def readSentiFeatures(sentilines):
    sentiDict = {}
    for line in sentilines: 
        comps = line.strip().split(',')
        sentiDict[comps[0]] = float(comps[1])
    return sentiDict

In [ ]:
meanSentiment = "../Final_features/median_sentiments.csv"
senti_csv = readFeatureFile(meanSentiment)

sentiDict = readSentiFeatures(senti_csv)

In [ ]:
postIds = sentiDict.keys()
print len(postIds)

In [ ]:
postUserMeta , finalDict = postFollowerMapping(postIds,totalPostDict,userDict)

In [ ]:
print len(postUserMeta)

In [ ]:
finalDict['1232413732706160640']

In [ ]:
finalDict['1275430531974504448']

In [ ]:
postUserMeta[100]

In [ ]:
# f = open(socialFeatures , 'w+')
# pickle.dump(postUserMeta , f)
# f.close()